<a href="https://colab.research.google.com/github/oraziotorre/MomentumShiftAI/blob/main/ModelsDevelopment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [14]:
# Importiamo il dataset ottenuto tramite le operazioni di Data Preprocessing
dataset = pd.read_csv("tennis_2010-now.csv")

for col in dataset.columns:
    print(col)

Unnamed: 0
match_id
Pt
Set1
Set2
Gm1
Gm2
Pts
Player1
Player2
Ranking1
Ranking2
Svr
PtWinner
PointType
SetID
PtSet
SetWinner
MatchWinner
p1_win_nobreak_point
p2_win_nobreak_point
p1_win_break_point
p2_win_break_point
p1_lost_nobreak_point
p2_lost_nobreak_point
p1_lost_break_point
p2_lost_break_point
serve_ace_1
serve_ace_2
serve_miss2_1
serve_miss1_2
rally_winner_1
rally_winner_2
rally_forced2_1
rally_forced1_2
rally_unforced1_2
rally_unforced2_1


In [15]:
dataset = dataset.drop(columns=['PointType'])#,'Player1','Player2','SetID', 'SetWinner','PtSet'
dataset

,Unnamed: 0,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Player1,Player2,...,serve_ace_1,serve_ace_2,serve_miss2_1,serve_miss1_2,rally_winner_1,rally_winner_2,rally_forced2_1,rally_forced1_2,rally_unforced1_2,rally_unforced2_1
0,0,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,1,0,0,0,0,0-0,106329,100644,...,0,0,0,0,0,0,0,0,0,1
1,1,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,2,0,0,0,0,0-15,106329,100644,...,0,0,0,0,0,0,0,1,0,0
2,2,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,3,0,0,0,0,0-30,106329,100644,...,1,0,0,0,0,0,0,0,0,0
3,3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,4,0,0,0,0,15-30,106329,100644,...,0,0,0,0,0,0,0,0,0,1
4,4,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,5,0,0,0,0,15-40,106329,100644,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710163,731734,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,119,0,1,3,5,AD-40,103819,103786,...,0,0,0,0,0,0,1,0,0,0
710164,731735,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,120,0,1,4,5,0-0,103819,103786,...,0,0,0,0,0,0,0,1,0,0
710165,731736,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,121,0,1,4,5,0-15,103819,103786,...,0,0,0,0,0,0,0,0,0,1
710166,731737,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,122,0,1,4,5,0-30,103819,103786,...,0,0,0,0,0,1,0,0,0,0


#**Train-Test split**

In [31]:
# Generazione delle partite inverse per aumentare la dimensione del dataset

def augment_with_symmetric(df: pd.DataFrame) -> pd.DataFrame:
    df_swapped = df.copy()

    # Colonne da swappare a coppie
    swap_pairs = [
        ('Set1', 'Set2'),
        ('Gm1', 'Gm2'),
        ('Player1', 'Player2'),
        ('Ranking1', 'Ranking2'),
        ('p1_win_nobreak_point', 'p2_win_nobreak_point'),
        ('p1_win_break_point', 'p2_win_break_point'),
        ('p1_lost_nobreak_point', 'p2_lost_nobreak_point'),
        ('p1_lost_break_point', 'p2_lost_break_point'),
        ('serve_ace_1', 'serve_ace_2'),
        ('serve_miss2_1', 'serve_miss1_2'),
        ('rally_winner_1', 'rally_winner_2'),
        ('rally_forced2_1', 'rally_forced1_2'),
        ('rally_unforced1_2', 'rally_unforced2_1')
    ]
    for col1, col2 in swap_pairs:
        df_swapped[[col1, col2]] = df[[col2, col1]].values

    # Colonne dove 1 <-> 2
    invert_1_2_cols = ['Svr', 'PtWinner', 'SetWinner', 'MatchWinner']
    for col in invert_1_2_cols:
        df_swapped[col] = df[col].replace({1: 2, 2: 1})

    # Inversione della colonna Pts (es. "40-15" -> "15-40")
    def invert_pts(value):
        try:
            left, right = value.split('-')
            return f"{right}-{left}"
        except:
            return value  # Se non è nel formato previsto, lo lascia invariato

    if 'Pts' in df.columns:
        df_swapped['Pts'] = df['Pts'].apply(invert_pts)

    # Aggiunta del suffisso 'simm' a match_id e set_id, se presenti
    for col in ['match_id', 'set_id']:
        if col in df_swapped.columns:
            df_swapped[col] = df_swapped[col].astype(str) + '_simm'

    return pd.concat([df, df_swapped], ignore_index=True)

In [47]:
# 1️⃣ Scaling e Normalizzazione

def preprocess_data(train_data, test_data):
    # Copie di sicurezza
    train_data = train_data.copy()
    test_data = test_data.copy()

    # === 1. Colonne da trasformare ===
    minmax_cols = ['Set1', 'Set2', 'Gm1', 'Gm2']
    standard_cols = ['Ranking1', 'Ranking2']
    categorical_col = 'Pts'
    onehot_col = 'Svr'

    # === 2. Altre feature da conservare (numeriche/categoriche già pronte) ===
    exclude_cols = minmax_cols + standard_cols + [categorical_col, onehot_col] + \
                   ['match_id', 'SetID', 'Player1', 'Player2', 'Pt', 'SetWinner', 'MatchWinner']

    other_features = [col for col in train_data.columns if col not in exclude_cols]

    # === 3. Normalizzazione ===
    minmax_scaler = MinMaxScaler()
    train_data[minmax_cols] = minmax_scaler.fit_transform(train_data[minmax_cols])
    test_data[minmax_cols] = minmax_scaler.transform(test_data[minmax_cols])

    # === 4. Standardizzazione ===
    standard_scaler = StandardScaler()
    train_data[standard_cols] = standard_scaler.fit_transform(train_data[standard_cols])
    test_data[standard_cols] = standard_scaler.transform(test_data[standard_cols])

    # === 5. One-hot encoding per la colonna 'Pts' ===
    pts_categories = train_data[categorical_col].unique()

    def one_hot_encode_column(df, col_name, categories, prefix):
        dummies = pd.get_dummies(df[col_name], prefix=prefix)
        for cat in categories:
            col = f'{prefix}_{cat}'
            if col not in dummies:
                dummies[col] = 0
        return dummies.sort_index(axis=1)

    train_pts = one_hot_encode_column(train_data, categorical_col, pts_categories, 'Pts')
    test_pts = one_hot_encode_column(test_data, categorical_col, pts_categories, 'Pts')

    train_data = pd.concat([train_data.reset_index(drop=True), train_pts], axis=1).drop(columns=[categorical_col])
    test_data = pd.concat([test_data.reset_index(drop=True), test_pts], axis=1).drop(columns=[categorical_col])

    # === 6. One-hot encoding per la colonna 'Svr' con rinomina ===
    def one_hot_svr(df, all_values):
        dummies = pd.get_dummies(df[onehot_col], prefix=onehot_col)
        for val in all_values:
            col = f'{onehot_col}_{val}'
            if col not in dummies:
                dummies[col] = 0
        dummies.rename(columns={f'{onehot_col}_{val}': f'{onehot_col}{val}' for val in all_values}, inplace=True)
        df = df.drop(columns=[onehot_col]).reset_index(drop=True)
        return pd.concat([df, dummies.reset_index(drop=True)], axis=1)

    svr_values = [1, 2]  # O usa sorted(set(train_data['Svr']) | set(test_data['Svr']))
    train_data = one_hot_svr(train_data, svr_values)
    test_data = one_hot_svr(test_data, svr_values)

    # === 7. Definizione feature finali ===
    pts_cols = [f'Pts_{cat}' for cat in pts_categories]
    svr_cols = [f'Svr{val}' for val in svr_values]
    feature_cols = minmax_cols + standard_cols + pts_cols + svr_cols + other_features

    # === 8. Target: conversione [1,2] → [0,1] ===
    target_cols = ['PtWinner', 'SetWinner', 'MatchWinner']
    for col in target_cols:
        train_data = train_data[train_data[col].isin([1, 2])]
        test_data = test_data[test_data[col].isin([1, 2])]
        train_data[col] = train_data[col].astype(int) - 1
        test_data[col] = test_data[col].astype(int) - 1

    # === 9. Conversione finale a float ===
    train_data[feature_cols] = train_data[feature_cols]
    test_data[feature_cols] = test_data[feature_cols]
    train_data.to_csv('train_data.csv', index=False)
    test_data.to_csv('test_data.csv', index=False)
    return train_data, test_data, feature_cols

In [33]:
# 2️⃣ Creazione sequenze

def create_sequences(data, feature_cols, target_col='MatchWinner'):
    X_sequences = []
    y_labels = []
    lengths = []


    #################################### CAMBIARE NEL CASO SI VUOLE FARE PER I SET_ID ##################################################
    # Ordina i punti all'interno di ciascun match
    data = data.sort_values(['match_id', 'Pt']).reset_index(drop=True)

    for match_id, df_match in data.groupby('match_id'):
        if target_col not in df_match.columns:
            continue  # Salta se manca il target

        seq_data = df_match[feature_cols].to_numpy(dtype=np.float32)
        X_seq = torch.tensor(seq_data, dtype=torch.float32)
        X_sequences.append(X_seq)

        # Prende il target del primo punto del match (assunto costante per tutti)
        match_winner = int(df_match[target_col].iloc[0])
        y_labels.append(match_winner)
        lengths.append(len(df_match))

    if not X_sequences:
        raise ValueError("Nessuna sequenza valida trovata nei dati.")

    # Padding delle sequenze
    X_padded = pad_sequence(X_sequences, batch_first=True)
    y_tensor = torch.tensor(y_labels, dtype=torch.long)
    lengths = torch.tensor(lengths, dtype=torch.long)

    return X_padded, y_tensor, lengths

In [34]:
# 3️⃣ Pipeline completa + DataLoader

def prepare_datasets(train_data, test_data, target_col='MatchWinner', batch_size=32):
    # Preprocessing dati
    train_proc, test_proc, feature_cols = preprocess_data(train_data, test_data)

    # Creazione sequenze
    X_train, y_train, train_lengths = create_sequences(train_proc, feature_cols, target_col)
    X_test, y_test, test_lengths = create_sequences(test_proc, feature_cols, target_col)

    # Costruzione dei TensorDataset
    train_dataset = TensorDataset(X_train, y_train, train_lengths)
    test_dataset = TensorDataset(X_test, y_test, test_lengths)

    # Funzione per ordinare le sequenze per lunghezza (richiesto da LSTM)
    def collate_fn(batch):
        inputs, targets, lengths = zip(*batch)
        inputs = torch.stack(inputs)
        targets = torch.stack(targets)
        lengths = torch.stack(lengths)
        sorted_idx = torch.argsort(lengths, descending=True)
        return inputs[sorted_idx], targets[sorted_idx], lengths[sorted_idx]

    # DataLoader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    return train_loader, test_loader, feature_cols

In [48]:
# Estrai tutti gli ID unici delle partite
match_ids = dataset['match_id'].unique()

# Split degli ID in training e test (es. 80% train, 20% test)
train_ids, test_ids = train_test_split(match_ids, test_size=0.2, random_state=42)

# Split per match-id
train_data = dataset[dataset['match_id'].isin(train_ids)].reset_index(drop=True)
test_data = dataset[dataset['match_id'].isin(test_ids)].reset_index(drop=True)

#train_data = augment_with_symmetric(train_data)
#test_data = augment_with_symmetric(test_data)

train_loader, test_loader, feature_cols = prepare_datasets(train_data, test_data, target_col='MatchWinner', batch_size=32)

#**Model 1**

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=1, num_classes=2, dropout=0.2):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                            batch_first=True, dropout=dropout, bidirectional=False)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x, lengths):
        # Packed sequence per gestire sequenze di lunghezza variabile
        packed_input = nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=True)
        packed_output, (h_n, _) = self.lstm(packed_input)

        # Usa l'ultimo hidden state
        out = self.fc(h_n[-1])  # h_n shape: (num_layers, batch, hidden_size)
        return out

In [ ]:
def train_lstm(model, train_loader, test_loader, num_epochs=10, lr=1e-3, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for X, y, lengths in train_loader:
            X, y, lengths = X.to(device), y.to(device), lengths.to(device)

            optimizer.zero_grad()
            outputs = model(X, lengths)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {total_loss:.4f}")

        evaluate_lstm(model, test_loader, device=device)

In [ ]:
def evaluate_lstm(model, data_loader, device='cpu'):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for X, y, lengths in data_loader:
            X, lengths = X.to(device), lengths.to(device)
            outputs = model(X, lengths)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            y_true.extend(y.numpy())
            y_pred.extend(preds)

    acc = accuracy_score(y_true, y_pred)
    print(f"Accuracy: {acc:.4f}")
    return acc

NameError: name 'prepare_datasets' is not defined

In [ ]:

# Inizializza il modello
input_size = len(feature_cols)
model = LSTMClassifier(input_size=input_size)

# Allenamento
train_lstm(model, train_loader, test_loader, num_epochs=10)

# **Model 2**